[View in Colaboratory](https://colab.research.google.com/github/kyokicchi/kaggle_codes/blob/master/trainLGBmodel.ipynb)

In [1]:
!pip install lightgbm

    100% |████████████████████████████████| 1.1MB 16.1MB/s 


In [0]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
from sklearn.model_selection import train_test_split
import pickle

In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [7]:
%cd ../content/gdrive/My Drive/projects/
%ls

[Errno 2] No such file or directory: '../content/gdrive/My Drive/projects/'
/content/gdrive/My Drive/projects
model_LGB.sav  model_NN.h5  model_XGB.sav  user_train_n.csv


In [0]:
s_data = 'user_train_n.csv'
filename = 'model_LGB.sav'
s_col_tgt = 'transactionRevenue_sum'
l_col_drop = [s_col_tgt]

In [9]:
%%time
df_train = pd.read_csv(s_data, index_col = 0)

CPU times: user 4.65 s, sys: 471 ms, total: 5.12 s
Wall time: 5.36 s


In [10]:
df_train.columns

Index(['transactionRevenue_sum', 'transactionRevenue_size', 'hits_sum',
       'hits_min', 'hits_max', 'hits_mean', 'hits_median', 'visitNumber_sum',
       'visitNumber_min', 'visitNumber_max', 'visitNumber_mean',
       'visitNumber_median', 'pageviews_sum', 'pageviews_min', 'pageviews_max',
       'pageviews_mean', 'pageviews_median', 'weekend_mean', 'year_mean',
       'isMobile_mean', 'day_min', 'day_max', 'day_mean', 'day_median',
       'day_var', 'day_std', 'month_min', 'month_max', 'month_mean',
       'month_median', 'month_var', 'month_std', 'weekofyear_min',
       'weekofyear_max', 'weekofyear_mean', 'weekofyear_median',
       'weekofyear_var', 'weekofyear_std'],
      dtype='object')

In [12]:
%%time
df_y = df_train[s_col_tgt]
df_x = df_train.drop(l_col_drop, axis=1)

CPU times: user 50.9 ms, sys: 114 ms, total: 165 ms
Wall time: 164 ms


In [13]:
%%time
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y)
x_eval, x_valid, y_eval, y_valid = train_test_split(x_test, y_test)



CPU times: user 653 ms, sys: 36.9 ms, total: 690 ms
Wall time: 695 ms


In [0]:

params = {
    'learning_rate': 0.03,
    'n_estimators': 200,
    'num_leaves': 128,
    'subsample': 0.2217,
    'colsample_bytree': 0.6810,
    'min_split_gain': np.power(10.0, -4.9380),
    'reg_alpha': np.power(10.0, -3.2454),
    'reg_lambda': np.power(10.0, -4.8571),
    'min_child_weight': np.power(10.0, 2),
    'silent': True
}

model = lgb.LGBMRegressor(**params)



In [17]:
%%time
model.fit(x_train, 
          y_train,
          eval_set=[(x_train, y_train),(x_eval, y_eval)],
          eval_metric='rmse',
          early_stopping_rounds=100)

[1]	training's l2: 1.4418e+16	training's rmse: 1.20075e+08	valid_1's l2: 3.88585e+15	valid_1's rmse: 6.23366e+07
Training until validation scores don't improve for 100 rounds.
[2]	training's l2: 1.43695e+16	training's rmse: 1.19873e+08	valid_1's l2: 3.84733e+15	valid_1's rmse: 6.20269e+07
[3]	training's l2: 1.43271e+16	training's rmse: 1.19696e+08	valid_1's l2: 3.82717e+15	valid_1's rmse: 6.18642e+07
[4]	training's l2: 1.42943e+16	training's rmse: 1.19559e+08	valid_1's l2: 3.80659e+15	valid_1's rmse: 6.16975e+07
[5]	training's l2: 1.42484e+16	training's rmse: 1.19367e+08	valid_1's l2: 3.77258e+15	valid_1's rmse: 6.14213e+07
[6]	training's l2: 1.42095e+16	training's rmse: 1.19204e+08	valid_1's l2: 3.75768e+15	valid_1's rmse: 6.12999e+07
[7]	training's l2: 1.41825e+16	training's rmse: 1.1909e+08	valid_1's l2: 3.74215e+15	valid_1's rmse: 6.11731e+07
[8]	training's l2: 1.41448e+16	training's rmse: 1.18932e+08	valid_1's l2: 3.71301e+15	valid_1's rmse: 6.09345e+07
[9]	training's l2: 1.4107e+

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.681,
       importance_type='split', learning_rate=0.03, max_depth=-1,
       min_child_samples=20, min_child_weight=100.0,
       min_split_gain=1.153453257821093e-05, n_estimators=200, n_jobs=-1,
       num_leaves=128, objective=None, random_state=None,
       reg_alpha=0.0005683292391383383, reg_lambda=1.3896326196366016e-05,
       silent=True, subsample=0.2217, subsample_for_bin=200000,
       subsample_freq=0)

In [18]:
%%time
pickle.dump(model, open(filename, 'wb'))

# files.download(filename) 

CPU times: user 53.5 ms, sys: 957 µs, total: 54.5 ms
Wall time: 674 ms
